# Planification d'Emplois du Temps Universitaires

## Introduction

Ce projet vise à appliquer les méthodes de programmation par contraintes (CSP) pour résoudre le problème de la planification des emplois du temps universitaires. Nous utiliserons un solveur CSP pour générer des emplois du temps valides en tenant compte de diverses contraintes.

## Modélisation du Problème

### Variables
- Créneaux horaires
- Salles
- Cours/Examens
- Enseignants

### Contraintes
- Exclusion mutuelle
- Capacité des salles
- Disponibilité des enseignants
- Préférences

## Implémentation avec un Solveur CSP

### Choix du Solveur
- MiniZinc ou OR-Tools

### Installation et Configuration

In [1]:
# Exemple d'installation du solveur
#!pip install ortools

from ortools.sat.python import cp_model

### Définition des données du problèmes

In [2]:
# Le nom des matières et le nombre d'heures hebdomadaires requis par matière
matieres = {
    "Histoire": 4,
    "Maths": 6,
    "Physique-Chimie": 6,
    "Philosophie": 4,
    "Sport": 3,
    "Anglais": 3,
    "Espagnol": 2,
    "Maths expertes": 2
}

# Créneaux horaires disponibles

jours_str = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
nb_jours = len(jours_str)

heures_str = ["8h30-9h30", "9h30-10h30", "10h30-11h30", "11h30-12h30", "14h-15h", "15h-16h", "16h-17h", "17h-18h"]
nb_heures = len(heures_str)

# Nombre de classes et salles
nb_classes = 3
nb_salles = 3


### Création du modèle CSP

In [ ]:
model = cp_model.CpModel()

### Création des variables:
- Matières
- Créneaux horaires (jours et heures)
- Classes
- Salle

In [ ]:
# x[cours, jour, heure, classe, salle] = 1 si le cours est programmé à ce créneau, sinon 0
x = {}
for nom_matiere in matieres:
    for jour in range(nb_jours):
        for heure in range(nb_heures):
            for classe in range(nb_classes):
                for salle in range(nb_salles):
                    x[(nom_matiere, jour, heure, classe, salle)] = model.NewBoolVar(
                        f'{nom_matiere}_{jour}_{heure}_classe{classe}_salle{salle}')

### Définition des contraintes
1. Respecter le nombre d'heures par matière par semaine
2. Une classe ne peut pas avoir deux cours en même temps
3. Une salle ne peut pas être utilisées par deux cours en même temps
4. Une matière ne peut pas être enseigné dans deux classes en même temps (car un prof par matière)
5. Pas plus de deux heures de cours d'affilé
6. Un cours de deux heures doit prendre place dans la même salle peandant les deux heures

In [ ]:
# 1. Chaque classe doit avoir le nombre d'nb_heures requis par semaine pour chaque cours
for nom_matiere, heures_matiere in matieres.items():
    for classe in range(nb_classes):
        model.Add(sum(x[(nom_matiere, jour, heure, classe, salle)]
                      for jour in range(nb_jours)
                      for heure in range(nb_heures)
                      for salle in range(nb_salles)) == heures_matiere)

# 2. Une classe ne peut pas avoir deux cours en même temps
for jour in range(nb_jours):
    for heure in range(nb_heures):
        for classe in range(nb_classes):
            model.Add(sum(x[(nom_matiere, jour, heure, classe, salle)]
                          for nom_matiere in matieres
                          for salle in range(nb_salles)) <= 1)

# 3. Une salle ne peut pas être utilisée par deux cours en même temps
for jour in range(nb_jours):
    for heure in range(nb_heures):
        for salle in range(nb_salles):
            model.Add(
                sum(x[(nom_matiere, jour, heure, classe, salle)]
                    for nom_matiere in matieres
                    for classe in range(nb_classes)) <= 1
            )

# 4. Un même cours ne peut pas être enseigné dans deux classes différentes au même moment
# (pour représenter la disponibilité des enseignants)
for nom_matiere in matieres:
    for jour in range(nb_jours):
        for heure in range(nb_heures):
            model.Add(sum(x[(nom_matiere, jour, heure, classe, salle)]
                          for classe in range(nb_classes)
                          for salle in range(nb_salles)) <= 1)

# 5. Pas plus de deux nb_heures d'un même cours à la suite pour une classe
for nom_matiere in matieres:
    for jour in range(nb_jours):
        for i in range(nb_heures - 2):
            for classe in range(nb_classes):
                model.Add(
                    sum(x[(nom_matiere, jour, i, classe, salle)] for salle in range(nb_salles)) +
                    sum(x[(nom_matiere, jour, i + 1, classe, salle)] for salle in range(nb_salles)) +
                    sum(x[(nom_matiere, jour, i + 2, classe, salle)] for salle in range(nb_salles)) < 3
                )

# 6. Pas de cours de deux heures dans une salle differente
for nom_matiere in matieres:
    for jour in range(nb_jours):
        for heure in range(nb_heures - 1):
            for classe in range(nb_classes):
                for salle1 in range(nb_salles):
                    for salle2 in range(nb_salles):
                        # Si au moins un des deux cours n'existe pas, la condition est validée
                        if salle1 != salle2:
                            model.Add(
                                x[(nom_matiere, jour, heure, classe, salle1)] +
                                x[(nom_matiere, jour, heure + 1, classe, salle2)] <= 1
                            )

### Minimisation
6. Eviter les trous dans l'emploi du temps pour les élèves
7. Eviter les cours de fin de journée (de 17h à 18h)

In [ ]:
# 7. Minimiser les "trous" pour les élèves

# On créé Y[c, j, h] = 1 si la classe c a un cours (de n'importe quelle matière) le jour j à l'heure h.
Y = {}
for classe in range(nb_classes):
    for jour in range(nb_jours):
        for heure in range(nb_heures):
            Y[(classe, jour, heure)] = model.NewBoolVar(f"Y_classe{classe}_{jour}_{heure}")
            model.Add(Y[(classe, jour, heure)] ==
                sum(x[(nom_matiere, jour, heure, classe, salle)]
                    for nom_matiere in matieres
                    for salle in range(nb_salles)
                    )
                )


trous = []
for classe in range(nb_classes):
    for jour in range(nb_jours):
        # Pour les trous d'une heure
        # On créé trou1 = pattern "cours - pas cours - cours"
        for heure in range(nb_heures - 2):
            trou1 = model.NewBoolVar(f"trou1_classe{classe}_{jour}_{heure}")
            # Assure que trou != 1 si la classe :
            # - n'a pas cours a l'heure h 
            # - OU a cours à l'heure h+1 
            # - OU n'a pas cours a l'heure h+2 
            model.Add(trou1 <= Y[(classe, jour, heure)])
            model.Add(trou1 <= 1 - Y[(classe, jour, heure + 1)])
            model.Add(trou1 <= Y[(classe, jour, heure + 2)])
            # Assure que trou = 1 si la classe :
            # - a cours a l'heure h
            # - ET n'a pas cours à l'heure h+1 
            # - ET a cours a l'heure h+2
            model.Add(trou1 >= Y[(classe, jour, heure)]
                      + (1 - Y[(classe, jour, heure + 1)])
                      + Y[(classe, jour, heure + 2)] - 2)
            trous.append(trou1)

        # Pour les trous de deux heures
        # On créé trou2 = pattern "cours - pas cours - pas cours - cours"
        for heure in range(nb_heures - 3):
            trou2 = model.NewBoolVar(f"trou2_classe{classe}_{jour}_{heure}")
            # Même logique en prenant deux heures de trous de plus en compte
            model.Add(trou2 <= Y[(classe, jour, heure)])
            model.Add(trou2 <= 1 - Y[(classe, jour, heure + 1)])
            model.Add(trou2 <= 1 - Y[(classe, jour, heure + 2)])
            model.Add(trou2 <= Y[(classe, jour, heure + 3)])
            model.Add(trou2 >= Y[(classe, jour, heure)]
                      + (1 - Y[(classe, jour, heure + 1)])
                      + (1 - Y[(classe, jour, heure + 2)])
                      + Y[(classe, jour, heure + 3)] - 3)
            trous.append(trou2)

            # Pour les trous de trois heures
            # On créé trou3 = pattern "cours - pas cours - pas cours - pas cours - cours"
            for heure in range(nb_heures - 4):
                trou3 = model.NewBoolVar(f"trou3_classe{classe}_{jour}_{heure}")
                # Même logique en prenant trois heures de trous de plus en compte
                model.Add(trou3 <= Y[(classe, jour, heure)])
                model.Add(trou3 <= 1 - Y[(classe, jour, heure + 1)])
                model.Add(trou3 <= 1 - Y[(classe, jour, heure + 2)])
                model.Add(trou3 <= 1 - Y[(classe, jour, heure + 3)])
                model.Add(trou3 <= Y[(classe, jour, heure + 4)])
                model.Add(trou3 >= Y[(classe, jour, heure)]
                          + (1 - Y[(classe, jour, heure + 1)])
                          + (1 - Y[(classe, jour, heure + 2)])
                          + (1 - Y[(classe, jour, heure + 3)])
                          + Y[(classe, jour, heure + 4)] - 4)
                trous.append(trou3)

# 8. Minimiser les cours après 17h
# On met un coût pour chaque cours planifié après 17h
cours_tardifs = []
dernier_cours = nb_heures - 1  # 17h00-18h00
for classe in range(nb_classes):
    for nom_matiere in matieres:
        for jour in range(nb_jours):
            for salle in range(nb_salles):
                ct = model.NewBoolVar(f"cours_tardif_classe{classe}_{nom_matiere}_{jour}")
                model.Add(ct == x[(nom_matiere, jour, dernier_cours, classe, salle)])
                cours_tardifs.append(ct)

# 9. Minimiser les cous d'une heure et maximiser les cours de deux heures
two_hour_courses = []
single_hour_courses = []
for classe in range(nb_classes):
    for nom_matiere in matieres:
        for jour in range(nb_jours):
            for heure in range(nb_heures - 1):  # Permet de vérifier les paires d'heures
                # Variable pour cours de deux heures consécutives
                two_hour_course = model.NewBoolVar(f"two_hour_course_{classe}_{nom_matiere}_{jour}_{heure}")

                # Vérifier si le cours a lieu sur deux heures consécutives
                model.Add(two_hour_course <= sum(x[(nom_matiere, jour, heure, classe, salle)]
                                                 for salle in range(nb_salles)))
                model.Add(two_hour_course <= sum(x[(nom_matiere, jour, heure + 1, classe, salle)]
                                                 for salle in range(nb_salles)))
                model.Add(two_hour_course >= sum(x[(nom_matiere, jour, heure, classe, salle)]
                                                 for salle in range(nb_salles))
                          + sum(x[(nom_matiere, jour, heure + 1, classe, salle)]
                                for salle in range(nb_salles)) - 1)

                two_hour_courses.append(two_hour_course)

            # Variables pour cours d'une seule heure
            for heure in range(nb_heures):
                # Cours d'une seule heure
                single_hour_course = model.NewBoolVar(f"single_hour_course_{classe}_{nom_matiere}_{jour}_{heure}")

                # Un cours d'une heure est un cours qui n'a pas de cours de la même matière
                # ni avant ni après
                cours_heure_courante = sum(x[(nom_matiere, jour, heure, classe, salle)]
                                           for salle in range(nb_salles))

                # Pour la première heure
                if heure == 0:
                    model.Add(single_hour_course <= cours_heure_courante)
                    model.Add(single_hour_course <= 1 - sum(x[(nom_matiere, jour, heure + 1, classe, salle)]
                                                            for salle in range(nb_salles)))

                # Pour la dernière heure
                elif heure == nb_heures - 1:
                    model.Add(single_hour_course <= cours_heure_courante)
                    model.Add(single_hour_course <= 1 - sum(x[(nom_matiere, jour, heure - 1, classe, salle)]
                                                            for salle in range(nb_salles)))

                # Pour les heures du milieu
                else:
                    model.Add(single_hour_course <= cours_heure_courante)
                    model.Add(single_hour_course <= 1 - sum(x[(nom_matiere, jour, heure - 1, classe, salle)]
                                                            for salle in range(nb_salles)))
                    model.Add(single_hour_course <= 1 - sum(x[(nom_matiere, jour, heure + 1, classe, salle)]
                                                            for salle in range(nb_salles)))

                single_hour_courses.append(single_hour_course)

#### On minimise
- La priorité est de réduire le nombre de trous (chaque trou = 10)
- Ensuite, on veut éviter les cours après 17h (coût = 1)
- Puis on minimiser les cours cours d'une seule heure et maximiser les cours de deux heures


In [ ]:
model.Minimize(10 * sum(trous) + 5 * sum(cours_tardifs) + 3 * sum(single_hour_courses) - sum(two_hour_courses))

## Résolution du Problème

### Exécution du Solveur

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30.0
status = solver.Solve(model)


### Analyse des Solutions
- Analyser et visualiser les solutions générées.

## Optimisation

### Critères d'Optimisation
- Minimiser les conflits
- Maximiser la satisfaction des préférences

### Comparaison des Solutions
- Avant et après optimisation

## Extensions et Tests

### Contraintes Supplémentaires
- Intégrer des règles de répartition géographique ou des préférences spécifiques.

### Tests avec Données Réelles
- Utiliser des données réelles pour tester la robustesse du modèle.

## Conclusion

- Résumé des résultats obtenus.
- Perspectives pour des améliorations futures.